## Import libraries and load data

In [ ]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error as mse
import itertools
from pandas.tseries.offsets import DateOffset, MonthEnd
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")


In [ ]:
# load data
data = pd.read_excel('5y_HE_forecast.xlsx', parse_dates=True)

In [ ]:
# select subset (Termination or Hiring) and resample it to month level
row_sel = data.loc[data['Action Type'] != 'Termination']
col_sel = row_sel[['Start Date', 'Personnel Number']]
col_sel = col_sel.set_index(['Start Date'])
df = col_sel.resample('M').count()

## Divide data into Training and Testing

In [ ]:
# total observation: 66; use 56 for training and 10 for testing
train, test = df.iloc[:56], df.iloc[56:]

## Explore data

In [ ]:
# a log transform can be used to lower the rate at which rolling mean increases and flatten out exponential change back to a linear relationship
df_log = np.log(df)
# plot data
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 2))
ax1.plot(df)
ax1.set_title('original')
ax2.plot(df_log)
ax2.set_title('log')
plt.show()

In [ ]:
# plot distribution of employee counts and check if data conforms to a Gaussian distribution
fig_d, (axd1, axd2) = plt.subplots(1, 2, figsize=(12, 2))
axd1.hist(df['Personnel Number'])
axd1.set_title('original')
axd2.hist(df_log['Personnel Number'])
axd2.set_title('log')
plt.show()


### Use rolling statistics and Augmented Dickey Fuller test to check stationarity
Conclusion from the test below: p-value is small on df_diff (I1), d is likely to be 1 in ARIMA

In [ ]:
# write a function to check stationarity
def check_stationarity(timeseries):
    # rolling statistics
    rolling_mean = timeseries.rolling(window=12).mean()
    rolling_std = timeseries.rolling(window=12).std()
    # rolling statistics plot
    original = plt.plot(timeseries, color='blue', label='Original')
    mean = plt.plot(rolling_mean, color='red', label='Rolling Mean')
    std = plt.plot(rolling_std, color='black', label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Statistics')
    plt.show(block=False)
    # Dickey–Fuller test:
    result = adfuller(timeseries['Personnel Number'])
    print('ADF Statistic: {}'.format(result[0]))
    print('p-value: {}'.format(result[1]))
    print('Critical Values:')
    for key, value in result[4].items():
        print('\t{}: {}'.format(key, value))


In [ ]:
# original df
check_stationarity(df)


In [ ]:
# df_log
check_stationarity(df_log)

In [ ]:
# df_diff
df_diff = df.diff(1)
check_stationarity(df_diff[1:])

### Use Auto Correlation Function and Partial Auto Correlation Function to check stationarity
Conclusion from the test below: both ACF and PACF show sudden decay at lag 1, the original data seems fairly stationary

In [ ]:
# ACF -> MA(q), PACF -> AR(p)
acf = plot_acf(df['Personnel Number'], lags=40)
pacf = plot_pacf(df['Personnel Number'], lags=40)


## Find the best hyperparameters by grid searching
The function takes two parameters, training dataset and an eval metric (aic or bic). 
- This first part loops though all the variations of parameters (for order and seasonal_order), and collects that into a dictionary.
- The second part finds the best hyperparameters from the dictionary and then fits the model. 
- The output is a dictionary with the model, aic, bic, order tuple and seasonal_order tuple.

In [ ]:
def find_best_sarima(train, eval_metric):
    
    p = d = q = range(0, 2)
    pdq = list(itertools.product(p, d, q))
    seasonal_pdq = [(x[0], x[1], x[2], 12) for x in pdq]

    counter = 0
    resultDict = {}
    
    for param in pdq:
        for param_seasonal in seasonal_pdq:
            try:
                counter += 1
                mod = sm.tsa.statespace.SARIMAX(train,
                                                order=param,
                                                seasonal_order=param_seasonal,
                                                enforce_stationarity=False,
                                                enforce_invertibility=False)

                results = mod.fit()
                resultDict[counter] = [results.aic, results.bic, param, param_seasonal]

            except:
                continue
                
    dict_to_df = pd.DataFrame.from_dict(resultDict, orient='index')
    
    if eval_metric == 'aic':
        best_run = dict_to_df[dict_to_df[0] == dict_to_df[0].min()].index.values
        best_run = best_run[0]
    elif eval_metric == 'bic':
        best_run = dict_to_df[dict_to_df[1] == dict_to_df[1].min()].index.values
        best_run = best_run[0]
            
    model = sm.tsa.statespace.SARIMAX(train,
                                      order=resultDict[best_run][2],
                                      seasonal_order=resultDict[best_run][3],
                                      enforce_stationarity=False,
                                      enforce_invertibility=False).fit()
    
    best_param = {'model':model, 
                  'aic':model.aic,
                  'bic':model.bic,
                  'order':resultDict[best_run][2], 
                  'seasonal_order':resultDict[best_run][3]}
    
    return best_param

In [ ]:
# call function and show the result
best = find_best_sarima(train, 'aic')
best

In [ ]:
# generate prediction using the model from the best dictionary
pred = best['model'].predict(start=test.index[0], end=test.index[-1], dynamic=True)
df['Forecast'] = pred

In [ ]:
# check mean squared error
mse(test, pred)

In [ ]:
def plot_test(pred):
    # plot the prediction with the actuals
    plt.figure(figsize=(16, 8))
    plt.plot(train.index, train, label='Train')
    plt.plot(pred.index, pred, label='SARIMA', color='r')
    plt.plot(test.index, test, label='Test', color='k')
    plt.legend(loc='best', fontsize='large')
    return plt.show()

In [ ]:
plot_test(pred)

### Try with manually input orders
Conclusion: although mse seems slightly lower on this set of hyperparameters, the result produced by grid search seems better visually.

In [ ]:
# fit the model
mod_manual = sm.tsa.statespace.SARIMAX(train,
                                      order=(0,0,1),
                                      seasonal_order=(3,0,1,12),
                                      enforce_stationarity=False,
                                      enforce_invertibility=False).fit()

In [ ]:
# predict
pred2 = mod_manual.predict(start=test.index[0], end=test.index[-1], dynamic=True)

In [ ]:
# check mean squared error
mse(test, pred2)

In [ ]:
plot_test(pred2)

## Produce the forecast result

In [ ]:
# create future dates
future_dates = [df.index[-1] + pd.offsets.MonthEnd(x) for x in range(0, 15)]
future_dates_df = pd.DataFrame(index=future_dates[1:], columns=df.columns)


In [ ]:
# concat 2 dfs
forecast_df = pd.concat([df, future_dates_df])

In [ ]:
# fit the model and create forecast
forecast = best['model'].forecast(steps=24, dynamic=True)
forecast_df['Forecast'] = forecast


In [ ]:
# plot the forecast
plt.figure(figsize=(16, 8))
plt.plot(forecast_df.index, forecast_df['Personnel Number'], label='Real_number')
plt.plot(forecast.index, forecast, label='Forecast', color='orange')
plt.legend(loc='best', fontsize='large')
plt.show()

In [ ]:
# print out forecast
forecast_df.loc[forecast_df['Personnel Number'].isnull()]

### -- The application ends --
### The following is for experimenting on the grid search and printing out the results. 
Beyond range(0, 3) it could take a long time to run.

In [ ]:
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in pdq]

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(train,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()
            
            print('ARIMA{}x{}12 - AIC:{} - BIC:{}'.format(param, 
                                                          param_seasonal, 
                                                          results.aic, 
                                                          results.bic))

        except:
            continue

In [ ]:
results.aic.min()